In [2]:
import os
import pandas as pd
import numpy as np
import networkx as nx

In [3]:
base_path = "C:/Users/brade/Downloads/millionsongsubset/MillionSongSubset"

In [4]:
def collect_song_paths(base_path):
    song_paths = []
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith('.h5'):
                song_paths.append(os.path.join(root, file))
    return song_paths

In [5]:
song_paths = collect_song_paths(base_path)
print(f"total number of songs: {len(song_paths)}")

total number of songs: 10000


In [11]:
data = {
    'user': ['user1', 'user1', 'user1', 'user2', 'user3'],  # Now length is 5
    'song': ['The Prisoner\s Song', 'Careless Love Blues', 'Looking My Love', 'Wreck of the Old 97', 'West Coast Blues']  # Length is 5
}

df = pd.DataFrame(data)

df_additional = pd.DataFrame({
    'user': ['user2', 'user3', 'user4', 'user1'],
    'song': ['Looking My Love', 'Careless Love Blues', 'Wreck of the Old 97', 'West Coast Blues']
})

df = pd.concat([df, df_additional], ignore_index=True)




In [ ]:
G = nx.Graph()
G.add_nodes_from(df['user'].unique(), bipartite=0) #users
G.add_nodes_from(df['song'].unique(), bipartite=1) #songs
G.add_edges_from(df.values)

In [ ]:
def katz_similarity(graph, beta=0.05):
    adjacency = nx.adjacency_matrix(graph).todense()
    I = np.identity(adjacency.shape[0])
    katz_matrix = np.linalg.inv(I - beta * adjacency) - I
    return katz_matrix

In [14]:
def normalize_scores(scores):
    max_score = max(scores.values())
    min_score = min(scores.values())
    return {k: (v - min_score) / (max_score - min_score) for k, v in scores.items()}

In [18]:
katz_sim = katz_similarity(G, beta=0.05)
nodes = list(G.nodes())
target_user = 'user1'
user_idx = node_index.index(target_user)

user_katz_similarities = katz_sim[user_idx]

# Convert to song scores
song_recommendations = {
    nodes[i]: user_katz_similarities[0, i]
    for i in range(len(nodes))
    if nodes[i] not in df[df['user'] == target_user]['song'].values and nodes[i] != target_user
}

normalized_recom = normalize_scores(song_recommendations)


print("\nKatz Similarity Recommendations for user1:")
for song, score in sorted(song_recommendations.items(), key=lambda x: -x[1]):
    print(f"{song}: {score:.4f}")
    
    
G_weighted = nx.Graph()
for _, row in df.iterrows():
    if(G_weighted.has_edge(row['user'], row['song'])):
        G_weighted[row['user']][row['song']]['weight'] += 1
    else:
        G_weighted.add_edge(row['user'], row['song'], weight=1)

# Step 2: Adamic-Adar Similarity Calculation
print("\nAdamic-Adar Scores for user1 and songs:")
adamic_adar_scores = nx.adamic_adar_index(G_weighted, [(target_user, n) for n in G_weighted.nodes() if n.startswith('song') and n not in df[df['user'] == target_user]['song'].values])

for u, v, score in adamic_adar_scores:
    print(f"{v}: {score:.4f}")


Katz Similarity Recommendations for user1:
user3: 0.0051
user2: 0.0025
Wreck of the Old 97: 0.0001
user4: 0.0000

Adamic-Adar Scores for user1 and songs:


C:\Users\brade\AppData\Local\Temp\ipykernel_2892\3214436760.py:24: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adjacency = nx.adjacency_matrix(graph).todense()


In [2]:
hybrid_scores = {
    song: 0.7 * katz_score + 0.3 * adamic_score
    for song, katz_score in normalized_recom.items()
    for _, song_, adamic_score in adamic_adar_scores
    if song == song_
}
print("\nHybrid Recommendations for user1")
for song, score in sorted(hybrid_scores.items(), key=lambda x: -x[1]):
    print(f"{song}: {score: .4f}")

NameError: name 'normalized_recom' is not defined

NameError: name 'git' is not defined